In [27]:
# splitting the largest files from the wikipedia corpus to parallelize the nlp pipeline

from os.path import join
import pandas as pd
from constants import *

fname = 'dewiki_02.pickle'
fpath = join(ETL_PATH, fname)
df = pd.read_pickle(fpath)
split = 25*1000
split *= 2
dfs = [df[:1*split], 
       df[1*split:], 
       #df[1*split:2*split], 
       #df[2*split:3*split], 
       #df[3*split:]
      ]
length = sum([len(d) for d in dfs])
assert len(df) == length
df.shape, [d.shape for d in dfs]
fsplit = fpath.rsplit('.', 1)
for i, d in enumerate(dfs):
    f = "{}_{:d}.{}".format(fsplit[0], i, fsplit[1])
    print(f)
    d.to_pickle(f)

../../master_cloud/corpora/preprocessed/dewiki_02_0.pickle
../../master_cloud/corpora/preprocessed/dewiki_02_1.pickle
